In [1]:
import sys
sys.path.append('../..')

from common.toronto_api import TorontoOpenDataAPI
from common.data_processors import FerryDataProcessor
import ferry_tickets.src.ferry_analysis as an

import pandas as pd

In [2]:
# Initialize API client and get package metadata using id
client = TorontoOpenDataAPI('toronto-island-ferry-ticket-counts')
# Check resources in package
# client.show_resources_info()

In [3]:
# Get data from first listed resource, which is the active datastore
df = client.get_resource_data()
df.head()

,_id,Timestamp,Redemption Count,Sales Count
0,1,2024-11-29T11:30:00,23,25
1,2,2024-11-29T11:15:00,7,1
2,3,2024-11-29T11:00:00,5,8
3,4,2024-11-29T10:45:00,1,0
4,5,2024-11-29T10:30:00,28,22


In [4]:
# Process data using data processor
df = FerryDataProcessor().process_resource(df)
df.head()


latest:	2024-11-29
today:	2024-11-29


,_id,Timestamp,Redemption Count,Sales Count,datetimeTimestamp,isToday,isLatest
0,1,2024-11-29T11:30:00,23,25,2024-11-29 11:30:00,True,True
1,2,2024-11-29T11:15:00,7,1,2024-11-29 11:15:00,True,True
2,3,2024-11-29T11:00:00,5,8,2024-11-29 11:00:00,True,True
3,4,2024-11-29T10:45:00,1,0,2024-11-29 10:45:00,True,True
4,5,2024-11-29T10:30:00,28,22,2024-11-29 10:30:00,True,True


In [5]:
# Update saved csv
df.to_csv(
    '../data/processed/Toronto Island Ferry Ticket Counts.csv',
    header=True,
    index=False
)


---